In [129]:
# import libraries

import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [181]:
df = pd.read_csv('../input/imdbmoviedata/IMDB-Movie-Data.csv')
df.head(5)

In [131]:
df.shape

In [132]:
df.describe()

In [183]:
# extract important columns

df_extracted = df[['Rank','Title','Genre','Year','Votes']].copy()
df_extracted['Genre'] = df_extracted.Genre.str.split(',')
df_extracted.head()

In [185]:
# split Genre columns into different columns based on the number of genres related to each movie

for index, row in df_extracted.iterrows():
    for genre in row['Genre']:
        df_extracted.at[index, genre] = 1

df_extracted.fillna(0,inplace=True)

In [188]:
# based on the user ratings he/she gave for the movies
# we determine to what extent the user likes what genres

user_movies = [
            {'Title':'Split', 'rating':5.6},
            {'Title':'Guardians of the Galaxy', 'rating':6},
            {'Title':'Sing', 'rating':6},
            {'Title':'La La Land', 'rating':8},
         ] 

df_user = pd.DataFrame(user_movies)
input_titles = df_user['Title'].to_list()

In [190]:
# extract the movies the user gave rating from the original dataframe

user_rated_movies = df_extracted[df_extracted['Title'].isin(input_titles)]
user_rated_movies = pd.merge(df_user, inputMovies)
user_rated_movies

In [195]:
# from previous dataframe, we take only genres to calculate the rating matrix
userGenreTable = user_rated_movies.drop('Rank', 1).drop('Title', 1).drop('Genre', 1).drop('Year', 1).drop('Votes',1).drop('rating',1)

In [196]:
# based on the user's preferences, we figure out how he/she likes certain genres
userProfile = userGenreTable.transpose().dot(user_rated_movies['rating'])
userProfile

In [197]:
df_extracted['Rating'] = df['Rating']

In [202]:
# after we do some calculations,we try to demonstrate movies with high recommendation  
genreTable = df_extracted.set_index(df_extracted['Rank'])
genreTable = genreTable.drop('Rank', 1).drop('Title', 1).drop('Genre', 1).drop('Year', 1)

recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

In [211]:
# The user give the number of recommendated movies and these movies are shown as a dataframe with all details

number_recom_movies = 50
recommendation =  df_extracted.loc[df_extracted['Rank'].isin(recommendationTable_df.head(number_recom_movies).keys())]
recommendation['recom_percentage'] = recommendationTable_df.copy()*100
recommendation = recommendation.sort_values(by='Rank')

In [212]:
recommendation = recommendation[['Rank','Title','Genre','Year','Votes','Rating','recom_percentage']]
recommendation['recom_percentage'] = recommendation['recom_percentage'].apply(lambda x: np.round(x,2))

In [214]:
# The final dataframe is ready

recommendation.head(10)

In [215]:
# save to csv file with recommended 50 movies as an example
recommendation.to_csv('recommendation_IMDB')